# 广告文案模型定制

使用广告文案数据来定制LLM，使得LLM能够根据服饰描述来自动生成广告文案。DashScope微调定制API的具体说明，可参见[参考文档](https://help.aliyun.com/zh/dashscope/developer-reference/quick-start-16)。

##  1. 安装DashScope Python SDK

In [1]:
!pip install  dashscope

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


1. 请点击链接[模型定制申请链接](https://dashscope.console.aliyun.com/model?spm=a2c4g.11186623.0.0.456e40e9480Mzq&applyModel=llm-fine-tune) 申请模型定制权限

2. 请打开[DashScope官网](https://dashscope.console.aliyun.com/apiKey)，获取你的DashScope API-KEY，并在下面代码块进行替换

In [8]:
# [Note: get your DashScope API key here first: https://dashscope.console.aliyun.com/apiKey]
import dashscope
dashscope.api_key='YOUR-DASHSCOPE-API-KEY'


## 2. 数据处理

### 首先，下载广告文案数据，并解压

In [1]:
!wget https://cherrytest-dataset.oss-cn-shanghai.aliyuncs.com/AdvertiseGen.tar.gz  && tar -zxf AdvertiseGen.tar.gz

--2023-09-08 19:34:52--  https://cherrytest-dataset.oss-cn-shanghai.aliyuncs.com/AdvertiseGen.tar.gz
Resolving cherrytest-dataset.oss-cn-shanghai.aliyuncs.com (cherrytest-dataset.oss-cn-shanghai.aliyuncs.com)... 47.101.83.176
Connecting to cherrytest-dataset.oss-cn-shanghai.aliyuncs.com (cherrytest-dataset.oss-cn-shanghai.aliyuncs.com)|47.101.83.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17069994 (16M) [application/gzip]
Saving to: ‘AdvertiseGen.tar.gz’

100%[======================================>] 17,069,994  70.3MB/s   in 0.2s   

2023-09-08 19:34:52 (70.3 MB/s) - ‘AdvertiseGen.tar.gz’ saved [17069994/17069994]



### 查看原数据格式

In [1]:
! cat AdvertiseGen/train.json |head -n 5

{"content": "类型#裤*版型#宽松*风格#性感*图案#线条*裤型#阔腿裤", "summary": "宽松的阔腿裤这两年真的吸粉不少，明星时尚达人的心头爱。毕竟好穿时尚，谁都能穿出腿长2米的效果宽松的裤腿，当然是遮肉小能手啊。上身随性自然不拘束，面料亲肤舒适贴身体验感棒棒哒。系带部分增加设计看点，还让单品的设计感更强。腿部线条若隐若现的，性感撩人。颜色敲温柔的，与裤子本身所呈现的风格有点反差萌。"}
{"content": "类型#裙*风格#简约*图案#条纹*图案#线条*图案#撞色*裙型#鱼尾裙*裙袖长#无袖", "summary": "圆形领口修饰脖颈线条，适合各种脸型，耐看有气质。无袖设计，尤显清凉，简约横条纹装饰，使得整身人鱼造型更为生动立体。加之撞色的鱼尾下摆，深邃富有诗意。收腰包臀,修饰女性身体曲线，结合别出心裁的鱼尾裙摆设计，勾勒出自然流畅的身体轮廓，展现了婀娜多姿的迷人姿态。"}
{"content": "类型#上衣*版型#宽松*颜色#粉红色*图案#字母*图案#文字*图案#线条*衣样式#卫衣*衣款式#不规则", "summary": "宽松的卫衣版型包裹着整个身材，宽大的衣身与身材形成鲜明的对比描绘出纤瘦的身形。下摆与袖口的不规则剪裁设计，彰显出时尚前卫的形态。被剪裁过的样式呈现出布条状自然地垂坠下来，别具有一番设计感。线条分明的字母样式有着花式的外观，棱角分明加上具有少女元气的枣红色十分有年轻活力感。粉红色的衣身把肌肤衬托得很白嫩又健康。"}
{"content": "类型#裙*版型#宽松*材质#雪纺*风格#清新*裙型#a字*裙长#连衣裙", "summary": "踩着轻盈的步伐享受在午后的和煦风中，让放松与惬意感为你免去一身的压力与束缚，仿佛要将灵魂也寄托在随风摇曳的雪纺连衣裙上，吐露出<UNK>微妙而又浪漫的清新之意。宽松的a字版型除了能够带来足够的空间，也能以上窄下宽的方式强化立体层次，携带出自然优雅的曼妙体验。"}
{"content": "类型#上衣*材质#棉*颜色#蓝色*风格#潮*衣样式#polo*衣领型#polo领*衣袖长#短袖*衣款式#拼接", "summary": "想要在人群中脱颖而出吗？那么最适合您的莫过于这款polo衫短袖，采用了经典的polo领口和柔软纯棉面料，让您紧跟时尚潮流。再配合上潮流的蓝色拼接设计，使您

DashScope模型定制化所需的训练数据格式如下：
<!-- ```json
{"prompt": "谁在文艺复兴时期绘制人体", "completion": "文艺复兴时期是一个关于艺术、文化和学术的复兴运动，在这个时期，许多艺术家都绘制了人体。"}
{"prompt": "I need a picture of someone crying.", "completion": "I'm sorry, but as an AI language model, I do not have the ability to display images."}
``` -->
```json
{"text": "\n\nHuman: 谁在文艺复兴时期绘制人体\n\nAssistant: 文艺复兴时期是一个关于艺术、文化和学术的复兴运动，在这个时期，许多艺术家都绘制了人体。"}
{"text": "\n\nHuman: I need a picture of someone crying.\n\nAssistant: I'm sorry, but as an AI language model, I do not have the ability to display images."}
```

我们编写代码，从下载的广告文案数据中，随机抽取200条作为训练数据, 100条作为测试数据，并将其转换为DashScope要求的数据格式。

In [6]:
import json
import random
def data_format_conversion(src_file, dst_file, num):
    with open(src_file, 'r') as infile:
        lines = infile.readlines()

    new_lines = random.choices(lines, k=num)
    with open(dst_file, 'w') as ofile:
        for l in new_lines:
            data = json.loads(l)
            ofile.write(json.dumps({'text': f"\n\nHuman: {data['content']}\n\nAssistant: {data['summary']}"}, ensure_ascii=False) + '\n')
        
raw_train_data = 'AdvertiseGen/train.json'
raw_test_data = 'AdvertiseGen/dev.json'
converted_train_data = 'AdvertiseGen/train_ds.jsonl'
converted_test_data = 'AdvertiseGen/dev_ds.jsonl'
data_format_conversion(raw_train_data, converted_train_data, 200)
data_format_conversion(raw_test_data, converted_test_data, 100)


### 查看转换后的训练集和验证集的数据格式

In [2]:
!cat AdvertiseGen/train_ds.jsonl | head -n 2

{"text": "\n\nHuman: 类型#上衣*风格#简约*风格#职场*图案#字母*图案#文字*图案#印花*图案#撞色*衣样式#棒球服*衣样式#外套*衣领型#v领*衣门襟#拉链*衣款式#拼接*衣款式#拉链\n\nAssistant: 个性的棒球服外套，让你在平时的工作中有一种轻松得户外free的感觉。立体的剪裁，挺括有型，营造干练职场范儿。门襟采用拉链设计，简约方便更显利落。字母印花与撞色的完美拼接，带有冲击性，表达你不凡的时尚态度。精致的v领剪裁，展露柔美的颈部曲线。"}
{"text": "\n\nHuman: 类型#上衣*颜色#纯色*风格#简约*风格#休闲*图案#纯色*图案#印花*衣样式#衬衫*衣样式#西装*衣领型#翻领*衣款式#纽扣\n\nAssistant: 以往的商务衬衫都给人以严肃、死板的套路感觉，但tedbaker的<UNK>作为英国著名衬衫专家，在衬衫设计上格外别出心裁。这款衬衫，精致的纽扣点缀其间，包括门襟、袖口，无处不彰显匠心。同时结合了小翻领的设计，时尚大气，彰显青春活力气息。简约大气的星星点点印花的点缀设计，使纯色跳脱出一份活力，既可以搭配西装，又能与休闲场合无缝衔接。"}
cat: write error: Broken pipe


In [3]:
!cat AdvertiseGen/dev_ds.jsonl | head -n 2

{"text": "\n\nHuman: 类型#裙*风格#复古*风格#文艺*图案#复古*图案#刺绣*裙型#百褶*裙款式#拼接*裙款式#钉珠\n\nAssistant: 这款丝绵上衣上有着精致的花朵刺绣装饰着，v型的领口设计看上去别致又显档次，裙摆与袖口处都是百褶样式的，穿在身上格外地飘逸吸睛。复古的钉珠与金线拼接，细节处的设计满满都是品味，让你轻松演绎复古的文艺范儿。"}
{"text": "\n\nHuman: 类型#裙*版型#宽松*版型#立体剪裁*风格#复古*图案#复古*图案#电影*图案#线条*裙长#连衣裙*裙衣门襟#系带\n\nAssistant: 想尝试电影里的日式和服，又担心<UNK>。试试这条简化改良版，日式风复古连衣裙。面料凉爽飘逸，前后都采用放大<UNK>的设计，延伸颈部线条。沉稳的网格设计，给人古风素雅的感觉。立体剪裁的宽松大袖子，十分有设计感，仿佛<UNK>自来。加上经典系带，系出优雅自信。"}


### 上传数据

In [9]:
from dashscope import File

resp = File.upload(converted_train_data, purpose='fine_tune', description='training data for ad')
if resp['status_code'] != 200:
    print('upload failed')
    print(resp)
else:
    train_file_id = resp['output']['uploaded_files'][0]['file_id']
    print(train_file_id)

4a70c68f-75dc-4794-bb5d-20264d8db755


In [10]:
resp = File.upload(converted_test_data, purpose='fine_tune', description='validation data for ad')
if resp['status_code'] != 200:
    print('upload failed')
    print(resp)
else:
    val_file_id = resp['output']['uploaded_files'][0]['file_id']
    print(val_file_id)

d3427abf-64ee-4193-84b0-bbde28734a34


## 3. 模型训练
### 启动训练

示例代码中我们使用chatglm-6b-v2进行训练，您也可以切换其他模型进行训练，详细的模型列表可以参考[支持定制的模型列表](https://help.aliyun.com/zh/dashscope/developer-reference/list-of-models-that-support-customization/?spm=a2c4g.11186623.0.0.456e40e9480Mzq)

In [25]:
from dashscope import FineTune

model = 'chatglm-6b-v2'

resp = FineTune.call(model=model,
              training_file_ids=[train_file_id],
              validation_file_ids=[val_file_id])
if resp['status_code'] != 200:
    print('finetune job run failed')
    print(resp)
else:
    print('job_id', resp['output']['job_id'])
    print('status', resp['output']['status'])


{'model': 'chatglm-6b-v2', 'training_file_ids': ['4a70c68f-75dc-4794-bb5d-20264d8db755'], 'validation_file_ids': ['d3427abf-64ee-4193-84b0-bbde28734a34'], 'hyper_parameters': {}}
finetune job run failed
{"status_code": 200, "request_id": "058c48d9-7792-9028-a891-1dfc62888cd8", "code": null, "message": "", "output": {"job_id": "ft-202309111655-9755", "status": "PENDING", "training_type": "sft", "create_time": "2023-09-11 16:55:17"}, "usage": null}


### 查看训练日志，等待训练完成

In [33]:
job_id = resp['output']['job_id']
events = FineTune.stream_events(job_id)
for e in events:
    if e['status_code'] != 200:
        print('finetune job failed')
        break
    elif 'finetune-job succeeded!' in e['output']:
        break
    else:
        print(e['output'])

connect success, now start to follow ft-202309111655-9755
2023-09-11 17:04:50,956 - INFO - finetune-job succeeded


### 训练结束后，获取训练模型名称

In [34]:
resp = FineTune.get(job_id)
if resp['output']['status'] == 'SUCCEEDED':
    finetuned_model = resp['output']['finetuned_output']
    print('finetuned_model:', finetuned_model)
else:
    print('finetune has not been finished')

finetuned_model: chatglm-6b-v2-ft-202309111655-9755


## 4. 模型部署
部署fine-tuned后的模型到服务，模型的部署需要一定的时间，约10分钟左右， 请耐心等待。

In [53]:
from dashscope import Deployment
import time
resp = Deployment.call(model=finetuned_model, capacity=1, suffix='aaa')
if resp['status_code'] != 200:
    print('deploy model failed')
    print(resp)
else:
    deployed_model = resp['output']['deployed_model']
    try:
        while True:  # wait for deployment ok.
            resp = dashscope.Deployment.get(deployed_model)
            if resp.status_code == 200:
                if resp.output['status'] in ['PENDING','DEPLOYING']:
                    time.sleep(30)
                    print('Deployment %s is %s' %
                            (deployed_model, resp.output['status']))
                else:
                    print('Deployment: %s status: %s' %
                            (deployed_model, resp.output['status']))
                    break

            else:
                print(resp)
    except Exception:
        print('You can get deployment status via: \
                dashscope deployments.get -d %s' % deployed_model)


{"status_code": 200, "request_id": "5a677313-b649-97c3-8d13-9dd345bc7be3", "code": null, "message": "", "output": {"deployed_model": "chatglm-6b-v2-ft-202309111655-9755-aaa", "status": "PENDING", "model_name": "chatglm-6b-v2-ft-202309111655-9755"}, "usage": null}
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309111655-9755-aaa is PENDING
Depl

## 5. 访问模型服务

当模型部署成功后，状态进入RUNNING。您可以通过提供新部署模型的id，来进行模型的推理调用。以下根据广告数据的格式提供prompt，来获取对应的生成广告文案。

In [54]:
from dashscope import Generation

resp = Generation.call(deployed_model, 
                        prompt = '类型#裙*风格#复古*风格#文艺*图案#复古*图案#刺绣*裙型#百褶*裙款式#拼接*裙款式#钉珠',
                        history=[])
print(resp)

{"status_code": 200, "request_id": "51799e6c-032c-9b68-82c5-8f38f2231362", "code": "", "message": "", "output": {"text": {"response": "这是一条带有复古和文艺气息的刺绣裙,具有百褶和钉珠装饰。裙型为A字型,裙款式为拼接和裙款式的设计,给您带来更加丰富多彩的感觉。适合各种场合,例如休闲、出游、商务和正式场合等。穿上它,一定会让您成为焦点。", "history": [["类型#裙*风格#复古*风格#文艺*图案#复古*图案#刺绣*裙型#百褶*裙款式#拼接*裙款式#钉珠", "这是一条带有复古和文艺气息的刺绣裙,具有百褶和钉珠装饰。裙型为A字型,裙款式为拼接和裙款式的设计,给您带来更加丰富多彩的感觉。适合各种场合,例如休闲、出游、商务和正式场合等。穿上它,一定会让您成为焦点。"]]}, "finish_reason": null, "choices": null}, "usage": {"input_tokens": 0, "output_tokens": 0}}


模型部署后， 通过如下代码（去掉注释）可删除已部署的模型(deployed_model)。

`注意`: 请慎重执行下面语句，执行后新部署的服务将被删除

In [47]:
from dashscope import Deployment
# Deployment.delete(deployed_model)

Deployed model: chatglm-6b-v2-ft-202309111124-c57c delete success
